In [ ]:
%pip install optuna

In [16]:
# do the hyperparameter search for the model
from config import Config
import optuna

base_config = Config.default_config()
base_config.final_module = 'lstm'
base_config.model = 'STGAT'

experiment_name = 'hp_optim'
config_dir = base_config.base_dir / 'configs' / experiment_name
config_dir.mkdir(exist_ok=True, parents=True)

base_config.log_base_dir = base_config.log_base_dir / experiment_name


In [17]:
from tensorboard.backend.event_processing import event_accumulator
import numpy as np

def get_min_eval_mse(cfg : Config):
    ea = event_accumulator.EventAccumulator(str(cfg.log_dir))
    ea.Reload()
    tags = ea.Tags()['scalars']
    mse_events = ea.Scalars('MSE/eval')
    mses = [event.value for event in mse_events]
    eval_steps = [event.step for event in mse_events]
    i_min_mse = np.argmin(mses)
    return mses[i_min_mse], i_min_mse
    

In [ ]:
import optuna

# minimize minimum validation mse over 100 epochs

# trial: optimize general learning parameters
def objective(trial: optuna.Trial):
    cfg = Config(**base_config.__dict__)
    cfg.name = f'experiment_{trial.number}'

    cfg.epochs = 60
    cfg.batch_size = trial.suggest_int('batch_size', 32, 512, log=True)
    cfg.optimizer_params['lr'] = trial.suggest_float('lr', 1e-5, 1e-1, log = True)
    cfg.optimizer_params['weight_decay'] = trial.suggest_float('weight_decay', 1e-5, 1e-2, log = True)
    cfg_path = str(config_dir / f'{cfg.name}.json')
    cfg.save_to_json(cfg_path)

    !python ./run_training.py --config_file $cfg_path
    min_mse, i_min_mse = get_min_eval_mse(cfg)
    print(f"Min mse {min_mse: .2e4} reached at {i_min_mse * cfg.eval_interval} / {cfg.epochs}")

    return min_mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)
    

[I 2024-11-30 15:46:30,404] A new study created in memory with name: no-name-66452fa9-e135-402c-80c9-916868536eb7


Processing...
/mnt/vol2/BikeSharePrediction/.venv/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
Processing dataset: 100%|█████████████████| 8908/8908 [00:03<00:00, 2847.89it/s]
Done!
/mnt/vol2/BikeSharePrediction/src/dataset.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future rel

[W 2024-11-30 15:46:57,724] Trial 0 failed with parameters: {'batch_size': 486, 'lr': 0.0007236753812494409, 'weight_decay': 6.344409887664863e-05} because of the following error: KeyError('Key MSE/eval was not found in Reservoir').
Traceback (most recent call last):
  File "/mnt/vol2/BikeSharePrediction/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_7209/3785574547.py", line 19, in objective
    min_mse, i_min_mse = get_min_eval_mse(cfg)
                         ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_7209/3705095441.py", line 8, in get_min_eval_mse
    mse_events = ea.Scalars('MSE/eval')
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/vol2/BikeSharePrediction/.venv/lib/python3.12/site-packages/tensorboard/backend/event_processing/event_accumulator.py", line 603, in Scalars
    return self.scalars.Items(tag)
           ^^^^^^^^^^^^^^^^^^^^^^^
  F

KeyError: 'Key MSE/eval was not found in Reservoir'